In [1]:
!pip install requests BeautifulSoup4

You are using pip version 9.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [2]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup

In [3]:
def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None


def is_good_response(resp):
    """
    Returns True if the response seems to be HTML, False otherwise.
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)


def log_error(e):
    """
    It is always a good idea to log errors. 
    This function just prints them, but you can
    make it do anything.
    """
    print(e)

In [4]:
raw_html = simple_get('https://altex.ro/telefon-samsung-galaxy-a10-32gb-2gb-ram-dual-sim-black/cpd/SMTA10BK/')
len(raw_html)

837297

In [5]:
html = BeautifulSoup(raw_html, 'html.parser')
html2 = html.find_all(attrs={"class": "Price-current"})[0].children
price = -1
for x in html2:
    y = x.string
    if(y != None and y.isdigit()):
        price = y

print(price)


649


In [6]:
def getIntPriceAltex(url):
    raw_html = simple_get(url)
    html = BeautifulSoup(raw_html, 'html.parser')
    html2 = html.find_all(attrs={"class": "Price-current"})[0].children
    price = -1
    for x in html2:
        y = x.string
        if(y != None):
            y = y.replace(".","")
            y = y.replace(",","")
        if(y != None and y.isdigit()):
            price = y

    return(price)


In [7]:
def getStringPriceF64(url):
    raw_html = simple_get(url)
    html = BeautifulSoup(raw_html, 'html.parser')
    html2 = html.find_all(attrs={"class": "skuBestPrice"})[0].string
    return html2

In [9]:
import time
from IPython.display import clear_output

produseAltex = [
    "https://altex.ro/telefon-huawei-y6-2019-32gb-2gb-ram-dual-sim-sapphire-blue/cpd/SMTY6DS19BL/",
    "https://altex.ro/telefon-myria-my9076-l500-16gb-2gb-ram-dual-sim-negru/cpd/SMTMY9076DSBK/",
    "https://altex.ro/telefon-huawei-p30-lite-128gb-4gb-ram-dual-sim-midnight-black/cpd/SMTP30LITEDSBK/"
]

produseF64 = [
    "https://www.f64.ro/nikon-speedlight-sb-700/p?idsku=132856"
]
prevPrice = {}
prevPrice["Altex"] = {}
prevPrice["F64"] = {}

for prod in produseAltex:
    prevPrice["Altex"][prod] = getIntPriceAltex(prod)

for prod in produseF64:
    prevPrice["F64"][prod] = getStringPriceF64(prod)
    
count = 0
while(1):
    count = count + 1
    clear_output()
    print("Seconds elapsed: ", count * 4)
    print()
    print("ALTEX")
    print("_______________")
    for prod in produseAltex:
        print()
        newPrice = getIntPriceAltex(prod)
        prod2 = prod.replace("https://altex.ro/", "")
        prod2 = prod2.split("/")[0].replace("-", " ")
        print("Pentru produsul", prod2,": ")
        print()
        if (newPrice != prevPrice["Altex"][prod]):
            prevPrice["Altex"][prod] = newPrice
            print("Pretul s-a schimbat de la ", prevPrice["Altex"][prod], " la ", newPrice)
        else:
            print("Pretul nu s-a schimbat de la ", prevPrice["Altex"][prod])
    print("F64")
    print("_______________")
    for prod in produseF64:
        print()
        prod2 = prod.replace("https://www.f64.ro/", "")
        prod2 = prod2.split("/")[0].replace("-", " ")
        print("Pentru produsul", prod2,": ")
        print()
        newPrice = getStringPriceF64(prod)
        if (newPrice != prevPrice["F64"][prod]):
            prevPrice["F64"][prod] = newPrice
            print("Pretul s-a schimbat de la ", prevPrice["F64"][prod], " la ", newPrice)
        else:
            print("Pretul nu s-a schimbat de la ", prevPrice["F64"][prod])
    time.sleep(4)

Seconds elapsed:  16

ALTEX
_______________

Pentru produsul telefon huawei y6 2019 32gb 2gb ram dual sim sapphire blue : 

Pretul nu s-a schimbat de la  549

Pentru produsul telefon myria my9076 l500 16gb 2gb ram dual sim negru : 

Pretul nu s-a schimbat de la  249

Pentru produsul telefon huawei p30 lite 128gb 4gb ram dual sim midnight black : 

Pretul nu s-a schimbat de la  999
F64
_______________

Pentru produsul nikon speedlight sb 700 : 

Pretul nu s-a schimbat de la  1.299,90 Lei


KeyboardInterrupt: 